# <img src="./resources/GA.png" width="25" height="25" /> <span style="color:Blue">DSI Capstone: Arizona/ Utah MTB Trail Recommender Engine</span> 
---
## <span style="color:Green">Cleaning / Merging Datasets / Preliminary EDA - Preprocessing</span>      

#### Ryan McDonald -General Assembly 

---

### Notebook Contents:

- [Reading the User Data](#intro)    
    - [User Data Cleaning](#cleanaz)
- [Reading the Trail Data](#trail)
    - [Trail Data Cleaning](#trailaz)
        - [Imputation/OHE](#imputeaz)
- [Export to CSV- Trails](#saveaz)

**Imports**

In [147]:
# general processing
import numpy as np
import pandas as pd
import re

# CSV manipulation
import os
import glob as glob

# for preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder 

# imputation of various nulls
from sklearn.impute import KNNImputer

A stretch goal for this project, post General Assembly, would be the inclusion of every state in the county. Scaling the data-scrapping for the rest of the country would take approximately 14 days at the rate of current scrapping. In order to ramp up efficiencies, the below cleaning blocks would be converted into a function to loop through each state's CSV file. I'm looking forward to putting this together at a greater scale! 

This data cleaning will produce two sets of cleaned data for each state.  One set will be for EDA and a user dashboard.  The other (with OHE categorical features and scaled data) will be for the recommender systems.

<a id='intro'></a>
## 1. Read the Clean Data - Arizona and Utah Users Data

The user data retreived from utilizing Octoparse API was not in the correct format.  For each trail name, all users (whether there was one or one hundred) were placed into one cell.  This cell also contained the average user rating.  The cell would need to be cleaned, reformated, and expanded into a new dataframe where each row would contain only one user and the trail they rated. For example, if 85 people rated 'Hiline Trail', then the subsequent dataframe would have 85 rows for 'Hiline Trail' with a unique user assinged to each.  This will be repeated for all trails with user ratings.

This user data will be utilized in a subsequent notebook to build a collaborative, binary-rating, user-based recommender system.

<a id='cleanaz'></a>
#### Arizona User Data

In [2]:
az_users = pd.read_csv('./data/user_data/arizona_trails_users.csv')
az_users.shape

(893, 2)

In [3]:
az_users.head()

,trail_name,user_name
0,Hiline Trail,Stars Average: 4.7 Maxx Byerl...
1,Slim Shady Trail,Stars Average: 4.4 Chase Roth...
2,Mescal,Stars Average: 4.6 Roger Coss...
3,Chuckwagon,Stars Average: 4.5 Roger Coss...
4,Tortolita Preserve Loop,Stars Average: 4.2 Joe Nelson...


<a id='cleanaz'></a>
#### Utah User Data

In [4]:
ut_users = pd.read_csv('./data/user_data/utah_trails_users.csv')
ut_users.shape

(929, 2)

In [5]:
ut_users.head()

,trail_name,user_name
0,Thunder Mountain Trail #33098,Stars Average: 4.7 ...
1,Wasatch Crest,Stars Average: 4.8 ...
2,Captain Ahab,Stars Average: 4.7 ...
3,Wire Mesa Loop,Stars Average: 4.6 ...
4,Ramblin',Stars Average: 4.7 ...



#### Cleaning Arizona / Utah 'user_name' Column

In [6]:
# cleaning function for Arizona Data

def user_clean(df):
    for each in range(0,len(df['trail_name'])):    
        # remove all the whitespace between names and replace with a comma
        s = re.sub('    \s+',',',df['user_name'][each])

        # remove the top two entries ('Stars' and 'Average')
        user_list = (s.split(","))[2:]

        # combine and create a list of tuples for users checked into each specific trail
        info = [(item, df['trail_name'][each]) for item in user_list]

        # building into a DataFrame of each user and trail check in 
        # trail_name now repeated when more than one check in present
        trail_checkins = pd.DataFrame(info, columns =['user_name', 'trail_name'])
    
        # saving each trail with check ins to a csv (to be concatenated below)
        trail_checkins.to_csv(f'./data/arizona_users/trail_{each}.csv', index = False)

    # what to choose files by
    extension = 'csv'

    # looping through the relative file path to grab all files
    all_filenames = [i for i in glob.glob('./data/arizona_users/*'.format(extension))]

    # combine all files in the list
    all_user = pd.concat([pd.read_csv(f) for f in all_filenames ])

    # export to new, concatenated csv
    # relative file path to were to save new csv
    all_user.to_csv( "./data/all_arizona_users.csv", index=False)

    # Getting a fresh index
    all_user.reset_index(inplace = True)
    
    return

In [7]:
user_clean(az_users)

In [8]:
# cleaning function for Utah Data

def user_clean(df):
    for each in range(0,len(df['trail_name'])):    
        # remove all the whitespace between names and replace with a comma
        s = re.sub('    \s+',',',df['user_name'][each])

        # remove the top two entries ('Stars' and 'Average')
        user_list = (s.split(","))[2:]

        # combine and create a list of tuples for users checked into each specific trail
        info = [(item, df['trail_name'][each]) for item in user_list]

        # building into a DataFrame of each user and trail check in 
        # trail_name now repeated when more than one check in present
        trail_checkins = pd.DataFrame(info, columns =['user_name', 'trail_name'])
    
        # saving each trail with check ins to a csv (to be concatenated below)
        trail_checkins.to_csv(f'./data/utah_users/trail_{each}.csv', index = False)

    # what to choose files by
    extension = 'csv'

    # looping through the relative file path to grab all files
    all_filenames = [i for i in glob.glob('./data/utah_users/*'.format(extension))]

    # combine all files in the list
    all_user = pd.concat([pd.read_csv(f) for f in all_filenames ])

    # export to new, concatenated csv
    # relative file path to were to save new csv
    all_user.to_csv( "./data/all_utah_users.csv", index=False)

    # Getting a fresh index
    all_user.reset_index(inplace = True)
    
    return

In [9]:
user_clean(ut_users)

In [10]:
# checking out the cleaned user data for Arizona
az_users = pd.read_csv( "./data/all_arizona_users.csv")
az_users.head()

,user_name,trail_name
0,Maxx Byerly,Hiline Trail
1,Cameron McFarland,Hiline Trail
2,Ascanio Pignatelli,Hiline Trail
3,Sabrina Katharina,Hiline Trail
4,Clayton Burtsfield,Hiline Trail


In [11]:
# verifying no nulls
az_users.shape, az_users.isnull().sum()

((5192, 2),
 user_name     0
 trail_name    0
 dtype: int64)

In [12]:
# checking out the cleaned user data for Utah
ut_users = pd.read_csv( "./data/all_utah_users.csv")
ut_users.head()

,user_name,trail_name
0,MadHamish H,Thunder Mountain Trail #33098
1,Matt Lane,Thunder Mountain Trail #33098
2,Phil Broadbent,Thunder Mountain Trail #33098
3,Jacob Crockett,Thunder Mountain Trail #33098
4,Heather Bond,Thunder Mountain Trail #33098


In [13]:
# verifying no nulls
ut_users.shape, ut_users.isnull().sum()

((7346, 2),
 user_name     0
 trail_name    0
 dtype: int64)

<a id='trail'></a>
## 2. Read and Clean Data - Arizona and Utah Trail Statistics

There was a wealth of data available for each trail scrapped. Various trail statisitcs thought most valuable to a rider are included in the datasets. The data retrieved was all object type, and included various extra characters/spaces/formating (mostly for user readability, but not good for modeling/filtering).  The below data will be cleaned and utilized in a dashboard where a user can choose a range of characteristics and the dashboard will only display those trails within the user-selected features. 

This data will also be utilized in a subsequent notebook for a feature-based content recommender system.

#### Reading in Arizona Trail Data

In [175]:
az = pd.read_csv('./data/raw_trail_data/arizona_trails.csv')
az.shape

(956, 21)

In [176]:
az.head()

,trail_name,length,difficulty,longitude,latitude,trail_link,city,popularity,rating,local_club,...,land_manager,land_manager_site,tot_climb,tot_descent,ave_grade,max_grade,max_elevation,min_elevation,dog_policy,e_bike_policy
0,Hiline Trail,3.1\n mi,VERY DIFFICULT,34.804602,-111.771244,https://www.mtbproject.com/trail/593534/hiline...,"Village of Oak Creek (Big Park), AZ",#1,4.7 (123),Verde Valley Cyclists Coalition,...,USFS - Coconino National Forest Office,http://www.fs.usda.gov/coconino,326',695',6%,35%,"4,646'","4,086'",Unknown,Unknown
1,Slim Shady Trail,2.6\n mi,INTERMEDIATE/DIFFICULT,34.790166,-111.765576,https://www.mtbproject.com/trail/593708/slim-s...,"Village of Oak Creek (Big Park), AZ",#2,4.4 (112),Verde Valley Cyclists Coalition,...,USFS - Coconino National Forest Office,http://www.fs.usda.gov/coconino,265',264',4%,11%,"4,464'","4,202'",Unknown,Unknown
2,Mescal,2.4\n mi,INTERMEDIATE/DIFFICULT,34.906801,-111.824568,https://www.mtbproject.com/trail/4960011/mescal,"West Sedona, AZ",#3,4.6 (63),Verde Valley Cyclists Coalition,...,USFS - Coconino National Forest Office,http://www.fs.usda.gov/coconino,206',166',3%,11%,"4,710'","4,520'",Unknown,Unknown
3,Chuckwagon,5.4\n mi,INTERMEDIATE,34.906632,-111.823948,https://www.mtbproject.com/trail/4960774/chuck...,"West Sedona, AZ",#4,4.5 (63),Verde Valley Cyclists Coalition,...,USFS - Coconino National Forest Office,http://www.fs.usda.gov/coconino,559',489',4%,13%,"4,679'","4,499'",Unknown,Unknown
4,Tortolita Preserve Loop,9.7\n mi,INTERMEDIATE,32.439082,-111.098523,https://www.mtbproject.com/trail/6390787/torto...,"Tortolita, AZ",#5,4.2 (66),Sonoran Desert Mountain Bicyclists,...,"Pima County - Natural Resources, Parks and Rec...",http://webcms.pima.gov/government/natural_reso...,520',520',2%,4%,"2,804'","2,326'",Unknown,Unknown


#### Reading in Utah Trail Data

In [177]:
ut = pd.read_csv('./data/raw_trail_data/utah_trails.csv')
ut.shape

(933, 21)

In [178]:
ut.head()

,trail_name,length,difficulty,longitude,latitude,trail_link,city,popularity,rating,local_club,...,land_manager,land_manager_site,tot_climb,tot_descent,ave_grade,max_grade,max_elevation,min_elevation,dog_policy,e_bike_policy
0,Thunder Mountain Trail #33098,8.1\n mi,INTERMEDIATE/DIFFICULT,37.702572,-112.279340,https://www.mtbproject.com/trail/1868735/thund...,"Panguitch, UT",#1,4.7 (87),Trail Alliance of Southern Utah (TASU),...,USFS - Dixie National Forest Office,http://www.fs.usda.gov/dixie,689',"1,681'",6%,27%,"8,263'","7,084'",Off-leash,Unknown
1,Wasatch Crest,12.5\n mi,INTERMEDIATE/DIFFICULT,40.613847,-111.570911,https://www.mtbproject.com/trail/3654027/wasat...,"Alta, UT",#2,4.8 (110),NaN,...,NaN,NaN,"1,084'","2,656'",6%,26%,"9,882'","7,616'",No Dogs,Not Allowed
2,Captain Ahab,4.2\n mi,DIFFICULT,38.518172,-109.618450,https://www.mtbproject.com/trail/648766/captai...,"Moab, UT",#3,4.7 (128),NaN,...,NaN,NaN,326',981',6%,23%,"4,898'","4,034'",Leashed,Not Allowed
3,Wire Mesa Loop,7.4\n mi,INTERMEDIATE/DIFFICULT,37.132839,-113.071259,https://www.mtbproject.com/trail/7021693/wire-...,"Springdale, UT",#4,4.6 (122),NaN,...,NaN,NaN,428',415',2%,12%,"4,502'","4,326'",Unknown,Allowed
4,Ramblin',3.3\n mi,INTERMEDIATE/DIFFICULT,38.637614,-109.784979,https://www.mtbproject.com/trail/7000569/ramblin,"Moab, UT",#5,4.7 (92),NaN,...,NaN,NaN,195',398',3%,12%,"5,279'","4,971'",Leashed,Not Allowed


<a id='trailaz'></a>
#### Arizona and Utah Trail Data Cleaning

**Extra Characters, DTypes, Scaling**

In [179]:
# trail data cleaning function

def trail_cleaning(df):
    # removing extra characters from 'length' and converting to float
    df['length']= (df['length'].str.replace('\n    mi','')).astype(float)

    # removing extra characters from 'city'
    df['city']= (df['city'].str.replace(', AZ',''))
    
    # setting text to lowercase for 'trail name'
    df['trail_name']=df['trail_name'].str.lower()
    
    # removing extra characters from 'rating' replacing 'nan' with 0, and converting to float
    df['rating'] = df['rating'].astype(str).str[:3].str.replace('nan','0').astype(float) 

    # removing '#' from 'arizona_rating' and scaling ranking from 0-1
    df['popularity']= (1-MinMaxScaler().fit_transform(df['popularity'].str.replace('#','').astype(float).values.reshape(-1,1)))

    # removing extra characters and converting to float for climb, descent, and elevation columns
    cols = ['tot_climb', 'tot_descent', 'max_elevation', 'min_elevation']
    # passing replacement and float type to each column
    df[cols] = df[cols].replace({",":"", "'":""}, regex=True).astype(float)

    # dropping '%' from ave and max_grade columns
    cols1 = ['ave_grade', 'max_grade']
    # passing replacement and float type to each column
    df[cols1] = df[cols1].replace({"%":""}, regex=True).astype(float)
    
    #dropping duplicate rows (trails names)
    df = df.drop_duplicates(subset='trail_name', keep='first', inplace= True)
    
    return

In [180]:
trail_cleaning(az)

In [181]:
trail_cleaning(ut)

In [182]:
# preserve an unscalled version of Arizona trail data for filtering dashboard 
az_trails = az.copy()
az_trails1 = az_trails.drop(['local_club_site', 'land_manager', 'land_manager_site', 'longitude', 'latitude'] , axis = 1)
az_trails1.to_csv( "./data/trail_dashboard/az_trails.csv", index=False)

# preserve an unscalled version of Utah trail data for filtering dashboard
ut_trails = ut.copy()
ut_trails1 = ut_trails.drop(['local_club_site', 'land_manager', 'land_manager_site', 'longitude', 'latitude'] , axis = 1)
ut_trails1.to_csv( "./data/trail_dashboard/ut_trails.csv", index=False)

<a id='imputeaz'></a>
#### Further Processing for Recommender System
**Imputing Nulls**

In [183]:
az_trails.isnull().sum()

trail_name             0
length                 0
difficulty             0
longitude             11
latitude              11
trail_link             0
city                   0
popularity             0
rating                 0
local_club           365
local_club_site      365
land_manager         622
land_manager_site    622
tot_climb             16
tot_descent           16
ave_grade             16
max_grade             16
max_elevation         16
min_elevation         16
dog_policy            27
e_bike_policy         16
dtype: int64

In [184]:
ut_trails.isnull().sum()

trail_name             0
length                 0
difficulty             0
longitude              4
latitude               4
trail_link             0
city                   0
popularity             0
rating                 0
local_club           408
local_club_site      408
land_manager         718
land_manager_site    718
tot_climb              0
tot_descent            0
ave_grade              0
max_grade              0
max_elevation          0
min_elevation          0
dog_policy            11
e_bike_policy          0
dtype: int64


#### Imputing club, land manager, dog and e-bike missing information with 'Unknown'

In [185]:
features=['local_club', 'local_club_site', 'land_manager', 'land_manager_site', 'dog_policy', 'e_bike_policy']
for i in features:
    az_trails[i].fillna('Unknown', inplace = True)
    ut_trails[i].fillna('Unknown', inplace = True)

#### Utilizing KNN Imputation for various missing trail statistics

This information is assumed 'Missing at Random'. The data is not systematically different from other points, and it is more likely that user-generated data just hasn't been collected yet. KNN Imputation should do a great job replacing missing data.

In [186]:
# scaling data to bring imputations onto a level playing field
features = ['longitude', 'latitude','tot_climb', 'tot_descent', 'ave_grade', 'max_grade', 'max_elevation', 'min_elevation','length', 'rating']
az_trails[features]= (MinMaxScaler().fit_transform(az[features]))
ut_trails[features]= (MinMaxScaler().fit_transform(ut[features]))

imp_features = ['tot_climb', 'tot_descent', 'ave_grade', 'max_grade', 'max_elevation', 'min_elevation']
# utilizing KNNImputer with 5 neighbors to fill in missing data for 'features'.
imputer = KNNImputer(n_neighbors=5)
az_trails[imp_features] = pd.DataFrame(imputer.fit_transform(az_trails[imp_features]))
ut_trails[imp_features] = pd.DataFrame(imputer.fit_transform(ut_trails[imp_features]))

In [187]:
# dropping columns not needed for recommender
column =['local_club','local_club_site','land_manager','land_manager_site', 'trail_link', 'city']
az_trails.drop(columns =column, inplace=True)
ut_trails.drop(columns =column, inplace=True)

**One Hot Encoding (pd.getdummies) 'city', 'dog_policy', 'e_bike_policy'**

In [188]:
# creating a list of features to convert to dummy columns
column=['difficulty', 'dog_policy', 'e_bike_policy']

dummies = pd.get_dummies(az_trails, columns=column)
az_trails = dummies
az_trails.columns = map(str.lower, az_trails.columns)
dummies = pd.get_dummies(ut_trails, columns=column)
ut_trails = dummies
ut_trails.columns = map(str.lower, ut_trails.columns)

### Checking in on the DataFrames

In [189]:
az_trails.head(2)

,trail_name,length,longitude,latitude,popularity,rating,tot_climb,tot_descent,ave_grade,max_grade,...,difficulty_intermediate,difficulty_intermediate/difficult,difficulty_very difficult,dog_policy_leashed,dog_policy_no dogs,dog_policy_off-leash,dog_policy_unknown,e_bike_policy_allowed,e_bike_policy_not allowed,e_bike_policy_unknown
0,hiline trail,0.022399,0.619678,0.507727,1.000000,0.94,0.022963,0.057739,0.315789,0.357143,...,0,0,1,0,0,0,1,0,0,1
1,slim shady trail,0.018786,0.617100,0.508725,0.998953,0.88,0.018666,0.021932,0.210526,0.112245,...,0,1,0,0,0,0,1,0,0,1


In [190]:
ut_trails.head(2)

,trail_name,length,longitude,latitude,popularity,rating,tot_climb,tot_descent,ave_grade,max_grade,...,difficulty_intermediate,difficulty_intermediate/difficult,difficulty_very difficult,dog_policy_leashed,dog_policy_no dogs,dog_policy_off-leash,dog_policy_unknown,e_bike_policy_allowed,e_bike_policy_not allowed,e_bike_policy_unknown
0,thunder mountain trail #33098,0.065165,0.140063,0.312952,1.000000,0.94,0.052217,0.148210,0.3,0.409091,...,0,1,0,0,0,1,0,0,0,1
1,wasatch crest,0.100563,0.726322,0.462306,0.998922,0.96,0.082152,0.234174,0.3,0.393939,...,0,1,0,0,1,0,0,0,1,0


<a id='saveaz'></a>
### Saving formatted Arizona Trials Dataframe for Modeling

In [191]:
az_trails.to_csv( "./data/recommender_data/az_trail_data.csv", index=False)
ut_trails.to_csv( "./data/recommender_data/ut_trail_data.csv", index=False)

In [ ]:
dd